In [1]:
!/usr/bin/python3 -m pip install --force-reinstall ./../../../target/wheels/biodivine_aeon-0.0.9a4-cp37-abi3-macosx_11_0_arm64.whl

Defaulting to user installation because normal site-packages is not writeable
Processing /Users/smijeva/PycharmProjects/biodivine-aeon-py/target/wheels/biodivine_aeon-0.0.9a4-cp37-abi3-macosx_11_0_arm64.whl
  Attempting uninstall: biodivine-aeon
    Found existing installation: biodivine_aeon 0.0.9a4
    Uninstalling biodivine_aeon-0.0.9a4:
      Successfully uninstalled biodivine_aeon-0.0.9a4

[notice] A new release of pip is available: 23.2.1 -> 24.0
[notice] To update, run: /Library/Developer/CommandLineTools/usr/bin/python3 -m pip install --upgrade pip


In [2]:
%pip install seaborn

Defaulting to user installation because normal site-packages is not writeable

[notice] A new release of pip is available: 23.2.1 -> 24.0
[notice] To update, run: /Library/Developer/CommandLineTools/usr/bin/python3 -m pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


## Demonstration of control on Myeloid model

The model we will observe is coming from Krumsiek, J., Marr, C., Schroeder, T., and Theis, F. J. (2011). Hierarchical differentiation of myeloid progenitors is encoded in the transcription factor network. PloS one, 6(8), e22649.

It describes myeloid cell differentiation into four possible types - erythrocytes, megakaryocytes, monocytes, and granulocytes.

![Myeloid_differentiation](myeloid.png)

In [3]:
import biodivine_aeon as ba
import seaborn as sns

In [4]:
# Load the model and create a Perturbation Graph

model = ba.BooleanNetwork.from_file('myeloid_witness.aeon')
pstg = ba.PerturbationGraph(model)

In [5]:
# Find attractors representing the four possible phenotypes

attractors = ba.find_attractors(pstg.as_original())

erythrocyte = pstg.fix_variable("EKLF", True).vertices()
erythrocyte_att = [a.vertices() for a in attractors if a.vertices().intersect(erythrocyte).cardinality() > 0.0][0]

megakaryocyte = pstg.fix_variable("Fli1", True).vertices()
megakaryocyte_att = [a.vertices() for a in attractors if a.vertices().intersect(megakaryocyte).cardinality() > 0.0][0]

monocyte = pstg.fix_variable("cJun", True).vertices()
monocyte_att = [a.vertices() for a in attractors if a.vertices().intersect(monocyte).cardinality() > 0.0][0]

granulocyte = pstg.fix_variable("Gfi1", True).vertices()
granulocyte_att = [a.vertices() for a in attractors if a.vertices().intersect(granulocyte).cardinality() > 0.0][0]

all_atts = [("Erythrocyte", erythrocyte_att), ("Megakaryocyte", megakaryocyte_att), ("Monocyte", monocyte_att), ("Granulocyte", granulocyte_att)]

In [7]:
results_one_step_min_control_size = {}
results_temporary_min_control_size = {}
results_permanent_min_control_size = {}

for source_att_name, source_att_state in all_atts:
    results_one_step_min_control_size[source_att_name] = {}
    results_temporary_min_control_size[source_att_name] = {}
    results_permanent_min_control_size[source_att_name] = {}
    for target_att_name, target_att_state in all_atts:
        if source_att_name == target_att_name:
            continue
        source_vec = []
        target_vec = []
        for v in model.variables():
            v_space = pstg.fix_variable(v, True).vertices()
            if source_att_state.intersect(v_space).cardinality() > 0.0:
                source_vec.append(True)
            else:
                source_vec.append(False)
                
            if target_att_state.intersect(v_space).cardinality() > 0.0:
                target_vec.append(True)
            else:
                target_vec.append(False)
                
        
        # Compute control map to query the results
        one_step_map = pstg.one_step_control(source_vec, target_vec, True)
        # Find the minimal control and store its size
        try:
            # one_step_min = len(one_step_map.working_perturbations(min_robustness=0.99, verbose=True,return_only_smallest=False)[0][0])
            one_step_min = one_step_map.working_perturbations(min_robustness=0.99, verbose=True,return_only_smallest=False)[0][0]
        except IndexError:
            one_step_min = -1
        results_one_step_min_control_size[source_att_name][target_att_name] = one_step_min

        temporary_map = pstg.temporary_control(source_vec, target_vec, False)
        try:
            # temporary_min = len(temporary_map.working_perturbations(min_robustness=0.99, verbose=False, return_only_smallest=False)[0][0])
            temporary_min = temporary_map.working_perturbations(min_robustness=0.99, verbose=False, return_only_smallest=False)[0][0]
        except IndexError:
            temporary_min = -1
            
        results_temporary_min_control_size[source_att_name][target_att_name] = temporary_min
        
        permanent_map = pstg.permanent_control(source_vec, target_vec, False)
        # Find the minimal control and store its size
        try:
            # permanent_min = len(permanent_map.working_perturbations(min_robustness=0.99, verbose=False, return_only_smallest=False)[0][0])
            permanent_min = permanent_map.working_perturbations(min_robustness=0.99, verbose=False, return_only_smallest=False)[0][0]
        except IndexError:
            permanent_min = -1
        results_permanent_min_control_size[source_att_name][target_att_name] = permanent_min
        
        print(source_att_name, target_att_name, one_step_min, temporary_min, permanent_min)

Erythrocyte Megakaryocyte {'EKLF': False, 'Fli1': True} {'EKLF': False} {'EKLF': False}
Erythrocyte Monocyte {'PU1': True} {'PU1': True} {'PU1': True}
Erythrocyte Granulocyte {'CEBPa': True, 'Gfi1': True, 'GATA1': False} {'CEBPa': True, 'EgrNab': False} {'EgrNab': False, 'CEBPa': True}
Megakaryocyte Erythrocyte {'EKLF': True, 'Fli1': False} {'EKLF': True} {'EKLF': True}
Megakaryocyte Monocyte {'PU1': True} {'PU1': True} {'PU1': True}
Megakaryocyte Granulocyte {'CEBPa': True, 'GATA1': False, 'Fli1': False, 'Gfi1': True} {'CEBPa': True, 'EgrNab': False} {'CEBPa': True, 'EgrNab': False}
Monocyte Erythrocyte {'GATA1': True, 'EKLF': True, 'PU1': False} {'GATA1': True, 'EKLF': True, 'GATA2': True} {'GATA1': True, 'EKLF': True, 'PU1': False}
Monocyte Megakaryocyte {'Fli1': True, 'PU1': False, 'GATA1': True} {'Fli1': True, 'PU1': False} {'PU1': False, 'Fli1': True}
Monocyte Granulocyte {'CEBPa': True, 'Gfi1': True, 'EgrNab': False} {'EgrNab': False, 'CEBPa': True} {'EgrNab': False, 'CEBPa': Tr

### MEGAKARYOCYTE TARGET

In [8]:
pstg.ceiled_phenotype_permanent_control(megakaryocyte_att, 5, "Forbidden", True, False).working_perturbations(0.99, False, False)

[({'Fli1': True, 'PU1': False}, ColorSet(cardinality=1))]

In [9]:
print(pstg.one_step_control(state_to_vec(model, erythrocyte_att, pstg), state_to_vec(model, megakaryocyte_att, pstg), True).working_perturbations(min_robustness=0.99, verbose=False, return_only_smallest=False))
print(pstg.one_step_control(state_to_vec(model, granulocyte_att, pstg), state_to_vec(model, megakaryocyte_att, pstg), True).working_perturbations(min_robustness=0.99, verbose=False, return_only_smallest=False))
print(pstg.one_step_control(state_to_vec(model, monocyte_att, pstg), state_to_vec(model, megakaryocyte_att, pstg), True).working_perturbations(min_robustness=0.99, verbose=False, return_only_smallest=False))
print(pstg.one_step_control(state_to_vec(model, megakaryocyte_att, pstg), state_to_vec(model, megakaryocyte_att, pstg), True).working_perturbations(min_robustness=0.99, verbose=False, return_only_smallest=False))

[({'Fli1': True, 'EKLF': False}, ColorSet(cardinality=1))]
[({'GATA1': True, 'CEBPa': False, 'PU1': False, 'Fli1': True}, ColorSet(cardinality=1))]
[({'PU1': False, 'Fli1': True, 'GATA1': True}, ColorSet(cardinality=1))]
[({}, ColorSet(cardinality=1))]


In [10]:
print(pstg.temporary_control(state_to_vec(model, erythrocyte_att, pstg), state_to_vec(model, megakaryocyte_att, pstg), True).working_perturbations(min_robustness=0.99, verbose=False, return_only_smallest=False))
print(pstg.temporary_control(state_to_vec(model, granulocyte_att, pstg), state_to_vec(model, megakaryocyte_att, pstg), True).working_perturbations(min_robustness=0.99, verbose=False, return_only_smallest=False))
print(pstg.temporary_control(state_to_vec(model, monocyte_att, pstg), state_to_vec(model, megakaryocyte_att, pstg), True).working_perturbations(min_robustness=0.99, verbose=False, return_only_smallest=False))
print(pstg.temporary_control(state_to_vec(model, megakaryocyte_att, pstg), state_to_vec(model, megakaryocyte_att, pstg), True).working_perturbations(min_robustness=0.99, verbose=False, return_only_smallest=False))


[({'EKLF': False}, ColorSet(cardinality=1)), ({'Fli1': True}, ColorSet(cardinality=1))]
[({'PU1': False, 'Fli1': True}, ColorSet(cardinality=1))]
[({'PU1': False, 'Fli1': True}, ColorSet(cardinality=1))]
[({}, ColorSet(cardinality=1))]


In [11]:
print(pstg.permanent_control(state_to_vec(model, erythrocyte_att, pstg), state_to_vec(model, megakaryocyte_att, pstg), True).working_perturbations(min_robustness=0.99, verbose=False, return_only_smallest=False))
print(pstg.permanent_control(state_to_vec(model, granulocyte_att, pstg), state_to_vec(model, megakaryocyte_att, pstg), True).working_perturbations(min_robustness=0.99, verbose=False, return_only_smallest=False))
print(pstg.permanent_control(state_to_vec(model, monocyte_att, pstg), state_to_vec(model, megakaryocyte_att, pstg), True).working_perturbations(min_robustness=0.99, verbose=False, return_only_smallest=False))
print(pstg.permanent_control(state_to_vec(model, megakaryocyte_att, pstg), state_to_vec(model, megakaryocyte_att, pstg), True).working_perturbations(min_robustness=0.99, verbose=False, return_only_smallest=False))


[({'EKLF': False}, ColorSet(cardinality=1)), ({'Fli1': True}, ColorSet(cardinality=1))]
[({'Fli1': True, 'PU1': False}, ColorSet(cardinality=1))]
[({'Fli1': True, 'PU1': False}, ColorSet(cardinality=1))]
[({}, ColorSet(cardinality=1))]


### ERYTHROCYTE TARGET

In [12]:
pstg.ceiled_phenotype_permanent_control(erythrocyte_att, 5, "Forbidden", True, False).working_perturbations(0.99, False, False)

[({'GATA1': True, 'Fli1': False, 'PU1': False}, ColorSet(cardinality=1)),
 ({'GATA1': True, 'PU1': False, 'EKLF': True}, ColorSet(cardinality=1))]

In [22]:
print(pstg.one_step_control(state_to_vec(model, erythrocyte_att, pstg), state_to_vec(model, erythrocyte_att, pstg), True).working_perturbations(min_robustness=0.99, verbose=False, return_only_smallest=False))
print(pstg.one_step_control(state_to_vec(model, granulocyte_att, pstg), state_to_vec(model, erythrocyte_att, pstg), True).working_perturbations(min_robustness=0.99, verbose=False, return_only_smallest=False))
print(pstg.one_step_control(state_to_vec(model, monocyte_att, pstg), state_to_vec(model, erythrocyte_att, pstg), True).working_perturbations(min_robustness=0.99, verbose=False, return_only_smallest=False))
print(pstg.one_step_control(state_to_vec(model, megakaryocyte_att, pstg), state_to_vec(model, erythrocyte_att, pstg), True).working_perturbations(min_robustness=0.99, verbose=False, return_only_smallest=False))


[({}, ColorSet(cardinality=1))]
[({'GATA1': True, 'CEBPa': False, 'PU1': False, 'EKLF': True}, ColorSet(cardinality=1))]
[({'EKLF': True, 'GATA1': True, 'PU1': False}, ColorSet(cardinality=1))]
[({'EKLF': True, 'Fli1': False}, ColorSet(cardinality=1))]


In [23]:
print(pstg.temporary_control(state_to_vec(model, erythrocyte_att, pstg), state_to_vec(model, erythrocyte_att, pstg), True).working_perturbations(min_robustness=0.99, verbose=False, return_only_smallest=False))
print(pstg.temporary_control(state_to_vec(model, granulocyte_att, pstg), state_to_vec(model, erythrocyte_att, pstg), True).working_perturbations(min_robustness=0.99, verbose=False, return_only_smallest=False))
print(pstg.temporary_control(state_to_vec(model, monocyte_att, pstg), state_to_vec(model, erythrocyte_att, pstg), True).working_perturbations(min_robustness=0.99, verbose=False, return_only_smallest=False))
print(pstg.temporary_control(state_to_vec(model, megakaryocyte_att, pstg), state_to_vec(model, erythrocyte_att, pstg), True).working_perturbations(min_robustness=0.99, verbose=False, return_only_smallest=False))


[({}, ColorSet(cardinality=1))]
[({'GATA1': True, 'GATA2': True, 'EKLF': True}, ColorSet(cardinality=1)), ({'PU1': False, 'GATA1': True, 'EKLF': True}, ColorSet(cardinality=1)), ({'EKLF': True, 'GATA2': True, 'PU1': False}, ColorSet(cardinality=1)), ({'Fli1': False, 'GATA1': True, 'GATA2': True}, ColorSet(cardinality=1)), ({'GATA1': True, 'Fli1': False, 'PU1': False}, ColorSet(cardinality=1)), ({'Fli1': False, 'GATA2': True, 'PU1': False}, ColorSet(cardinality=1))]
[({'EKLF': True, 'GATA1': True, 'GATA2': True}, ColorSet(cardinality=1)), ({'PU1': False, 'EKLF': True, 'GATA1': True}, ColorSet(cardinality=1)), ({'EKLF': True, 'PU1': False, 'GATA2': True}, ColorSet(cardinality=1)), ({'GATA1': True, 'Fli1': False, 'GATA2': True}, ColorSet(cardinality=1)), ({'Fli1': False, 'PU1': False, 'GATA1': True}, ColorSet(cardinality=1)), ({'PU1': False, 'Fli1': False, 'GATA2': True}, ColorSet(cardinality=1))]
[({'EKLF': True}, ColorSet(cardinality=1)), ({'Fli1': False}, ColorSet(cardinality=1))]


In [24]:
print(pstg.permanent_control(state_to_vec(model, erythrocyte_att, pstg), state_to_vec(model, erythrocyte_att, pstg), True).working_perturbations(min_robustness=0.99, verbose=False, return_only_smallest=False))
print(pstg.permanent_control(state_to_vec(model, granulocyte_att, pstg), state_to_vec(model, erythrocyte_att, pstg), True).working_perturbations(min_robustness=0.99, verbose=False, return_only_smallest=False))
print(pstg.permanent_control(state_to_vec(model, monocyte_att, pstg), state_to_vec(model, erythrocyte_att, pstg), True).working_perturbations(min_robustness=0.99, verbose=False, return_only_smallest=False))
print(pstg.permanent_control(state_to_vec(model, megakaryocyte_att, pstg), state_to_vec(model, erythrocyte_att, pstg), True).working_perturbations(min_robustness=0.99, verbose=False, return_only_smallest=False))


[({}, ColorSet(cardinality=1))]
[({'EKLF': True, 'GATA1': True, 'PU1': False}, ColorSet(cardinality=1)), ({'PU1': False, 'GATA1': True, 'Fli1': False}, ColorSet(cardinality=1))]
[({'EKLF': True, 'GATA1': True, 'PU1': False}, ColorSet(cardinality=1)), ({'Fli1': False, 'GATA1': True, 'PU1': False}, ColorSet(cardinality=1))]
[({'EKLF': True}, ColorSet(cardinality=1)), ({'Fli1': False}, ColorSet(cardinality=1))]


### MONOCYTE TARGET

In [25]:
pstg.ceiled_phenotype_permanent_control(monocyte_att, 5, "Forbidden", True, False).working_perturbations(0.99, False, False)

[({'PU1': True, 'CEBPa': False}, ColorSet(cardinality=1))]

In [26]:
print(pstg.one_step_control(state_to_vec(model, erythrocyte_att, pstg), state_to_vec(model, monocyte_att, pstg), True).working_perturbations(min_robustness=0.99, verbose=False, return_only_smallest=False))
print(pstg.one_step_control(state_to_vec(model, granulocyte_att, pstg), state_to_vec(model, monocyte_att, pstg), True).working_perturbations(min_robustness=0.99, verbose=False, return_only_smallest=False))
print(pstg.one_step_control(state_to_vec(model, monocyte_att, pstg), state_to_vec(model, monocyte_att, pstg), True).working_perturbations(min_robustness=0.99, verbose=False, return_only_smallest=False))
print(pstg.one_step_control(state_to_vec(model, megakaryocyte_att, pstg), state_to_vec(model, monocyte_att, pstg), True).working_perturbations(min_robustness=0.99, verbose=False, return_only_smallest=False))

[({'PU1': True}, ColorSet(cardinality=1))]
[({'CEBPa': False}, ColorSet(cardinality=1))]
[({}, ColorSet(cardinality=1))]
[({'PU1': True}, ColorSet(cardinality=1))]


In [27]:
print(pstg.temporary_control(state_to_vec(model, erythrocyte_att, pstg), state_to_vec(model, monocyte_att, pstg), True).working_perturbations(min_robustness=0.99, verbose=False, return_only_smallest=False))
print(pstg.temporary_control(state_to_vec(model, granulocyte_att, pstg), state_to_vec(model, monocyte_att, pstg), True).working_perturbations(min_robustness=0.99, verbose=False, return_only_smallest=False))
print(pstg.temporary_control(state_to_vec(model, monocyte_att, pstg), state_to_vec(model, monocyte_att, pstg), True).working_perturbations(min_robustness=0.99, verbose=False, return_only_smallest=False))
print(pstg.temporary_control(state_to_vec(model, megakaryocyte_att, pstg), state_to_vec(model, monocyte_att, pstg), True).working_perturbations(min_robustness=0.99, verbose=False, return_only_smallest=False))

[({'PU1': True}, ColorSet(cardinality=1))]
[({'CEBPa': False}, ColorSet(cardinality=1))]
[({}, ColorSet(cardinality=1))]
[({'PU1': True}, ColorSet(cardinality=1))]


In [28]:
print(pstg.permanent_control(state_to_vec(model, erythrocyte_att, pstg), state_to_vec(model, monocyte_att, pstg), True).working_perturbations(min_robustness=0.99, verbose=False, return_only_smallest=False))
print(pstg.permanent_control(state_to_vec(model, granulocyte_att, pstg), state_to_vec(model, monocyte_att, pstg), True).working_perturbations(min_robustness=0.99, verbose=False, return_only_smallest=False))
print(pstg.permanent_control(state_to_vec(model, monocyte_att, pstg), state_to_vec(model, monocyte_att, pstg), True).working_perturbations(min_robustness=0.99, verbose=False, return_only_smallest=False))
print(pstg.permanent_control(state_to_vec(model, megakaryocyte_att, pstg), state_to_vec(model, monocyte_att, pstg), True).working_perturbations(min_robustness=0.99, verbose=False, return_only_smallest=False))


[({'PU1': True}, ColorSet(cardinality=1))]
[({'CEBPa': False}, ColorSet(cardinality=1))]
[({}, ColorSet(cardinality=1))]
[({'PU1': True}, ColorSet(cardinality=1))]


### GRANULOCYTE TARGET

In [29]:
pstg.ceiled_phenotype_permanent_control(granulocyte_att, 5, "Forbidden", True, False).working_perturbations(0.99, False, False)

[({'CEBPa': True, 'cJun': False}, ColorSet(cardinality=1)),
 ({'Gfi1': True, 'CEBPa': True}, ColorSet(cardinality=1)),
 ({'CEBPa': True, 'EgrNab': False}, ColorSet(cardinality=1))]

In [30]:
print(pstg.one_step_control(state_to_vec(model, erythrocyte_att, pstg), state_to_vec(model, granulocyte_att, pstg), True).working_perturbations(min_robustness=0.99, verbose=False, return_only_smallest=False))
print(pstg.one_step_control(state_to_vec(model, granulocyte_att, pstg), state_to_vec(model, granulocyte_att, pstg), True).working_perturbations(min_robustness=0.99, verbose=False, return_only_smallest=False))
print(pstg.one_step_control(state_to_vec(model, monocyte_att, pstg), state_to_vec(model, granulocyte_att, pstg), True).working_perturbations(min_robustness=0.99, verbose=False, return_only_smallest=False))
print(pstg.one_step_control(state_to_vec(model, megakaryocyte_att, pstg), state_to_vec(model, granulocyte_att, pstg), True).working_perturbations(min_robustness=0.99, verbose=False, return_only_smallest=False))


[({'CEBPa': True, 'GATA1': False, 'Gfi1': True}, ColorSet(cardinality=1))]
[({}, ColorSet(cardinality=1))]
[({'CEBPa': True, 'EgrNab': False, 'Gfi1': True}, ColorSet(cardinality=1))]
[({'GATA1': False, 'CEBPa': True, 'Fli1': False, 'Gfi1': True}, ColorSet(cardinality=1)), ({'PU1': True, 'CEBPa': True, 'Gfi1': True, 'GATA1': False}, ColorSet(cardinality=1)), ({'PU1': True, 'SCL': False, 'CEBPa': True, 'Gfi1': True}, ColorSet(cardinality=1))]


In [31]:
print(pstg.temporary_control(state_to_vec(model, erythrocyte_att, pstg), state_to_vec(model, granulocyte_att, pstg), True).working_perturbations(min_robustness=0.99, verbose=False, return_only_smallest=False))
print(pstg.temporary_control(state_to_vec(model, granulocyte_att, pstg), state_to_vec(model, granulocyte_att, pstg), True).working_perturbations(min_robustness=0.99, verbose=False, return_only_smallest=False))
print(pstg.temporary_control(state_to_vec(model, monocyte_att, pstg), state_to_vec(model, granulocyte_att, pstg), True).working_perturbations(min_robustness=0.99, verbose=False, return_only_smallest=False))
print(pstg.temporary_control(state_to_vec(model, megakaryocyte_att, pstg), state_to_vec(model, granulocyte_att, pstg), True).working_perturbations(min_robustness=0.99, verbose=False, return_only_smallest=False))


[({'CEBPa': True, 'EgrNab': False}, ColorSet(cardinality=1)), ({'CEBPa': True, 'Gfi1': True}, ColorSet(cardinality=1)), ({'CEBPa': True, 'cJun': False}, ColorSet(cardinality=1))]
[({}, ColorSet(cardinality=1))]
[({'CEBPa': True, 'EgrNab': False}, ColorSet(cardinality=1)), ({'CEBPa': True, 'Gfi1': True}, ColorSet(cardinality=1)), ({'CEBPa': True, 'PU1': False}, ColorSet(cardinality=1)), ({'CEBPa': True, 'cJun': False}, ColorSet(cardinality=1))]
[({'CEBPa': True, 'EgrNab': False}, ColorSet(cardinality=1)), ({'Gfi1': True, 'CEBPa': True}, ColorSet(cardinality=1)), ({'CEBPa': True, 'cJun': False}, ColorSet(cardinality=1))]


In [32]:
print(pstg.permanent_control(state_to_vec(model, erythrocyte_att, pstg), state_to_vec(model, granulocyte_att, pstg), True).working_perturbations(min_robustness=0.99, verbose=False, return_only_smallest=False))
print(pstg.permanent_control(state_to_vec(model, granulocyte_att, pstg), state_to_vec(model, granulocyte_att, pstg), True).working_perturbations(min_robustness=0.99, verbose=False, return_only_smallest=False))
print(pstg.permanent_control(state_to_vec(model, monocyte_att, pstg), state_to_vec(model, granulocyte_att, pstg), True).working_perturbations(min_robustness=0.99, verbose=False, return_only_smallest=False))
print(pstg.permanent_control(state_to_vec(model, megakaryocyte_att, pstg), state_to_vec(model, granulocyte_att, pstg), True).working_perturbations(min_robustness=0.99, verbose=False, return_only_smallest=False))


[({'EgrNab': False, 'CEBPa': True}, ColorSet(cardinality=1)), ({'CEBPa': True, 'Gfi1': True}, ColorSet(cardinality=1)), ({'CEBPa': True, 'cJun': False}, ColorSet(cardinality=1))]
[({}, ColorSet(cardinality=1))]
[({'CEBPa': True, 'EgrNab': False}, ColorSet(cardinality=1)), ({'Gfi1': True, 'CEBPa': True}, ColorSet(cardinality=1)), ({'CEBPa': True, 'cJun': False}, ColorSet(cardinality=1))]
[({'EgrNab': False, 'CEBPa': True}, ColorSet(cardinality=1)), ({'Gfi1': True, 'CEBPa': True}, ColorSet(cardinality=1)), ({'CEBPa': True, 'cJun': False}, ColorSet(cardinality=1))]
